<a target="_blank" href="https://colab.research.google.com/github/jakegibb/nyphil-programming-analysis/blob/main/data_restructuring_and_analysis/nyphil_programming_analysis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Imports

Import necessary libaries and bring in New York Philharmonic JSON data.

In [1]:
import os
import json
import pandas as pd

# Import New York Phil Performance History from JSON file to a dictionary
perf_history = {}

with open('../nyphilarchive_performance_history/Programs/json/complete.json', 'r') as json_file:
    perf_history = json.load(json_file)

perf_history_records = perf_history['programs']

### Unnesting and Restructuring Data

This code unnests the JSON performance history, organizing the dataset around "works" instead of "programs."

In the final DataFrame, each row represents the performance of a single work on a particular NY Phil program. The date associated with the performance is the first date a program was performed.

In [2]:
# Function to return the earliest date from an array of concert information
def firstConcertDate(concert_array):
    concert_dates = []

    for concert in concert_array:
        concert_dates.append(concert['Date'])

    concert_dates.sort()
    earliest_date = concert_dates[0]

    return earliest_date

# Convert dictionary from JSON to a DataFrame
perf_df = pd.DataFrame(perf_history_records)

# Create new DataFrame by exploding works column, creating a new row for each work on a program
perf_df_works_exploded = (perf_df
                          .explode('works')
                          .reset_index()
                          )

# Split array of 'works' info into multiple columns and drop redundant fields
works_and_programs = perf_df_works_exploded[['programID', 'works']]
works_dict = works_and_programs.to_dict(orient='records')
works_df = (pd.json_normalize(works_dict, sep='_')
            .drop(['programID', 'works','works_soloists', 'works_conductorName'], axis=1)
            .fillna('')
            )

# Aggregate all 'movement' information into 'works_movement' column
works_df['works_movement__'] = works_df[['works_movement__', 'works_movement_em']].agg(' '.join, axis=1).str.strip()
works_df['works_movement'] = works_df[['works_movement', 'works_movement__']].agg(' '.join, axis=1).str.strip()

# Aggregate all 'workTitle' information into 'works_workTitle' column
works_df['works_workTitle__'] = works_df[['works_workTitle__', 'works_workTitle_em']].agg(' '.join, axis=1).str.strip()
works_df['works_workTitle'] = works_df[['works_workTitle', 'works_workTitle__']].agg(' '.join, axis=1).str.strip()

# Change column names to singular/lowercase
works_df_cleaned = (works_df
                    .rename(columns={
                        'works_ID' : "work_id",
                        "works_composerName" : "work_composer_name",
                        "works_workTitle" : "work_title",
                        "works_movement" : "work_movement"})
                    )

# Concatonate the exploded/cleaned 'performance' and 'works' DataFrames, renaming 'programID' to match the new naming convention
work_history_df = (pd
                   .concat([perf_df_works_exploded, works_df_cleaned], axis=1)
                   .rename(columns={'programID' : 'program_id'})
                   )

# Create a column ('date') that specifies the first date the work was performed on a concert for the associated program
work_history_df['date'] = work_history_df['concerts'].apply(firstConcertDate)

# Remove movement information from work_id
work_history_df['work_id'] = work_history_df['work_id'].apply(lambda x: x.split('*')[0])

### New Works Per Season

This code filters and counts New York Philharmonic's performance history to determine how many "new works" were performed each season.

A "new work" is defined as as piece that has is being performed by the ensemble for the first time in its history.

For comparison, the resulting table also includes the number of "total composers" performed on each season.

In [3]:
# Remove rows with no 'work_id'; remove intermission rows ('work_id' = '0'); set index to be a combination of 'work_id' and 'program_id'; reorder and select relevant columns.
work_history_df_cleaned = (
    work_history_df[(work_history_df['work_id'] != '') & (work_history_df['work_id'] != '0') & (work_history_df['work_id'] != '')]
    .set_index(['work_id', 'program_id'])
    .sort_values(['work_id', 'date'], ascending=[True, True])
    .loc[:,['date', 'season', 'work_title','work_movement','work_composer_name']]
    )

# Filter dataframe for only the first performance of a work
new_work_df = (work_history_df_cleaned
                .groupby('work_id')
                .first()
                .sort_values(by='date'))

# Count the number of new works performed in an orchestral season
new_works_per_season = new_work_df.season.value_counts()
new_works_per_season.sort_index(inplace=True)

#Count of total number of works performed per season
all_works = work_history_df[(work_history_df['work_id'] != '') 
                            & (work_history_df['work_id'] != '0')]
total_works_per_season = (all_works[['season', 'work_id']]
                        .groupby('season')
                        .nunique()
                        .rename(columns={'work_id' : 'total_works'}))

#Merge "new work" and "total work" counts into one dataframe
works_per_season_df = total_works_per_season.merge(new_works_per_season.rename('new_works'), left_index=True, right_index=True)

# Calculate number of "repeat compsers" per season
works_per_season_df['repeat_works'] = (works_per_season_df['total_works']
                                               - works_per_season_df['new_works'])

# Calculate percentage of season with new/repeat composers
works_per_season_df['new_works_p'] = (works_per_season_df['new_works'] 
                                              / works_per_season_df['total_works'])
works_per_season_df['repeat_works_p'] = (works_per_season_df['repeat_works']
                                              / works_per_season_df['total_works'])

# Add season start year for filtering
#works_per_season_df['season_year'] = works_per_season_df.index.str[:4].astype(int)

### New Composers Per Season

This code filters and counts New York Philharmonic's performance history to determine how many "new composers" were performed  each season.

A "new composer" is counted when the Philharmonic performs a work by a composer whose music has never been performed by the orchestra in its history.

For comparison, the resulting table also includes the number of "total composers" performed on each season.

In [4]:
# Remove rows with no composer history; change index to be a combination of composer name, work id, and program id; reorder and select relevant columns
composer_history_df_cleaned = (work_history_df[(work_history_df['work_composer_name'] != '')]
                            .set_index(['work_composer_name','work_id', 'program_id'])
                            .sort_values(['work_composer_name', 'date'])
                            .loc[:,['date','season','work_title','work_movement']]
    )

# Filter data frame for only the first time a composer is performed by the orchestra
new_composer_df = (composer_history_df_cleaned
                        .groupby('work_composer_name')
                        .first()
                        .sort_values(by='date'))

# Count the number of new composers performed in each season
new_composers_per_season = new_composer_df.season.value_counts()
new_composers_per_season.sort_index(inplace=True)

#Count of total number of composers performed per season
all_composers = work_history_df[(work_history_df['work_composer_name'] != '')]
total_composers_per_season = (all_composers[['season', 'work_composer_name']]
                        .groupby('season')
                        .nunique()
                        .rename(columns={'work_composer_name' : 'total_composers'}))

#Merge "new composer" and "total composer" counts into dataframe
composers_per_season_df = total_composers_per_season.merge(new_composers_per_season.rename('new_composers'), how='outer', on= 'season')

# Replace NaNs with 0
composers_per_season_df = composers_per_season_df.fillna(0)

# Calculate number of "repeat composers" per season
composers_per_season_df['repeat_composers'] = (composers_per_season_df['total_composers']
                                               - composers_per_season_df['new_composers'])

# Calculate percentage of season with new/repeat composers
composers_per_season_df['new_composers_p'] = (composers_per_season_df['new_composers'] 
                                              / composers_per_season_df['total_composers'])
composers_per_season_df['repeat_composers_p'] = (composers_per_season_df['repeat_composers']
                                              / composers_per_season_df['total_composers'])

composers_per_season_df.head(20)


,total_composers,new_composers,repeat_composers,new_composers_p,repeat_composers_p
season,,,,,
1842-43,16,16.0,0.0,1.000000,0.000000
1843-44,15,6.0,9.0,0.400000,0.600000
1844-45,15,5.0,10.0,0.333333,0.666667
1845-46,19,11.0,8.0,0.578947,0.421053
1846-47,17,4.0,13.0,0.235294,0.764706
1847-48,11,3.0,8.0,0.272727,0.727273
1848-49,18,6.0,12.0,0.333333,0.666667
1849-50,10,2.0,8.0,0.200000,0.800000
1850-51,18,9.0,9.0,0.500000,0.500000


### New Works and New Composers Combined DataFrames

In [5]:
# Merge new composer and new work dataframes
expansive_programming_df = composers_per_season_df.merge(works_per_season_df, how='outer', on= 'season')

expansive_programming_df['new_works_and_composers_p'] = (expansive_programming_df['new_composers']
                                               / expansive_programming_df['new_works'])

expansive_programming_df['season_year'] = expansive_programming_df.index.str[:4].astype(int)

# Bin season by decade. Decades are defined as ###0 – ###9 (e.g. 1980-1989).
decade_bins = []
number_of_bins = (2030-1840)//10 +1 # Range of decades in dataset
for x in range (number_of_bins):
    decade_bins.append(1839 + (10*x))

expansive_programming_df['decade_bin'] = pd.cut(x=expansive_programming_df['season_year'], bins=decade_bins)

expansive_programming_df['decade'] = expansive_programming_df['decade_bin'].astype(str).str[1:5].astype(int) + 1

# Export to CSV
expansive_programming_df.to_csv('nyphil_expansive_programming_data_by_season.csv')

expansive_programming_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182 entries, 1842-43 to 2023-24
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   total_composers            182 non-null    int64   
 1   new_composers              182 non-null    float64 
 2   repeat_composers           182 non-null    float64 
 3   new_composers_p            182 non-null    float64 
 4   repeat_composers_p         182 non-null    float64 
 5   total_works                182 non-null    int64   
 6   new_works                  182 non-null    int64   
 7   repeat_works               182 non-null    int64   
 8   new_works_p                182 non-null    float64 
 9   repeat_works_p             182 non-null    float64 
 10  new_works_and_composers_p  182 non-null    float64 
 11  season_year                182 non-null    int64   
 12  decade_bin                 182 non-null    category
 13  decade                     182